In [35]:
from match_functions import get_match_database
data = get_match_database('2025-03-01', '2025-03-15', leagues = ["Ligue 1"], save = True, add = False)

Extraction des liens...



100%|██████████| 15/15 [02:05<00:00,  8.38s/it]


Chargement de la base de données...
 
Extraction terminée en  4.52 minutes


In [38]:
import ace_tools_open as tools
tools.display_dataframe_to_user("Data", data)

Data


In [ ]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def get_page_rate(url):
    options = Options()
    options.add_argument("--headless")  # Mode headless activé
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=options)
    
    try:
        driver.get(url)

        # Attendre que les stats des joueurs apparaissent ET contiennent du texte
        WebDriverWait(driver, 15).until(
            lambda d: any(e.text.strip() for e in d.find_elements(By.CLASS_NAME, "player-stat-value"))
        )

        html = driver.page_source
    finally:
        driver.quit()

    soup = BeautifulSoup(html, "html.parser")

    return soup

def get_rate(url):
    soup = get_page_rate(url)

    # Extraction des notes
    rate = [c.text.strip() if c.text.strip() else "NaN" for c in soup.find_all("span", class_="player-stat-value")]

    # Extraction des noms des joueurs
    player = [c.text.strip() for c in soup.find_all("span", class_="player-name")][:len(rate)]

    # Extraction du nom du match
    match = re.search(r'202\d-202\d-(.+)', url)
    match_name = match.group(1) if match else "Unknown"
    
    # Team
    team1 = []

    # Création du DataFrame
    df_rate = pd.DataFrame({'Team1': [match_name] * len(player), 'Player': player, 'Rate': rate})

    return df_rate


In [15]:
# 🔥 Test
url = "https://fr.whoscored.com/matches/1824847/live/france-ligue-1-2024-2025-auxerre-strasbourg"
df = get_rate(url)
df

✅ Cookies acceptés


,Match,Player,Rate
0,auxerre-strasbourg,Oppegård,6.3
1,auxerre-strasbourg,Sinayoko,5.9
2,auxerre-strasbourg,Onaiwu,6.0
3,auxerre-strasbourg,Joly,6.0
4,auxerre-strasbourg,Massengo,6.2
5,auxerre-strasbourg,Ayé,NaN
6,auxerre-strasbourg,Percin,NaN
7,auxerre-strasbourg,Dioussé,NaN
8,auxerre-strasbourg,Diomandé,NaN
9,auxerre-strasbourg,Léon,6.3


In [16]:
soup = get_page_rate(url)

⚠️ Pas de cookies à accepter


In [67]:
def get_affluence(soup):
    code = soup.find_all("a", class_="team-link")
    team = [code[0].text, code[1].text]

    c = soup.find_all("div", class_="info-block cleared")[2].text.replace("Coup d'envoi:", "").replace("Date:Sun, ", "/").split("/")
    date_str = c[1]
    date_obj = datetime.strptime(date_str, '%d-%b-%y')  # Convertir en objet datetime
    formatted_date = date_obj.strftime('%d/%m/%Y')  # Reformater en "01/03/2025"
    date = [formatted_date, c[0]]

    affluence = [int(soup.find_all("span", class_="value")[1].text.replace(",", ""))]

    return date + team + affluence

In [68]:
get_affluence(soup)

['02/03/2025', '17:15', 'Auxerre', 'Strasbourg', 16495]